In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import warnings
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import platform

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [2]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE': 2048,
    'SEED':42
}

PATH = os.getcwd() + '/data/'
if platform.system() == 'Darwin':
    LOADPATH = '/Users/a1r/Desktop/DL/timeseries_new_data/'
else:
    LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [4]:
train_input = np.load(LOADPATH + 'train_input_minmax.npy')
train_target = np.load(LOADPATH + 'train_target_minmax.npy')

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [6]:
data_len = len(train_input)
train_dataset = CustomDataset(train_input[:-int(data_len*0.2)], train_target[:-int(data_len*0.2)])
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(train_input[-int(data_len*0.2):], train_target[-int(data_len*0.2):])
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### LSTM

In [7]:
class LSTM(nn.Module):
    def __init__(self, num_layers = 1, input_size=10, hidden_size=256, output_size=CFG['PREDICT_SIZE']):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers = self.num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )
            
        self.actv = nn.ReLU()
    
    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 9)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)
        
        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden)
        
        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))
        
        return output.squeeze(1)
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device))

In [8]:
def train(model, optimizer, train_loader, val_loader, device, best_loss = 9999999):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = best_loss
    train_loss = {}
    val_loss = {}
    
    # Epoch
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss_li = []
        train_mae = []
    
    # Iteration
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss_li.append(loss.item())
        
        val_loss_li = validation(model, val_loader, criterion, device)
        val_loss_mean = np.mean(val_loss_li)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss_li):.5f}] Val Loss : [{val_loss_mean:.5f}]')
        
        if best_loss > val_loss_mean:
            best_loss = val_loss_mean
            best_model = model
            print('Model Saved')

        train_loss[epoch] = train_loss_li
        val_loss[epoch] = val_loss_li

    return best_model, train_loss, val_loss, best_loss

In [9]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return val_loss

In [10]:
model = LSTM(num_layers=2, input_size = 9)
# model.load_state_dict(torch.load(os.getcwd() + '/model/lstm_2layers_1024B_1.pth'))
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model, train_loss, val_loss, best_loss = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/2005 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03107] Val Loss : [0.03118]
Model Saved


  0%|          | 0/2005 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

...

  0%|          | 0/502 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.01759] Val Loss : [0.01732]
Model Saved


  0%|          | 0/2005 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.01754] Val Loss : [0.01784]


In [11]:
SAVE_PATH = os.getcwd() + '/model/lstm_2layers_2048B_1.pth'
torch.save(infer_model.state_dict(), SAVE_PATH)

train_loss = np.array(train_loss)
val_loss = np.array(val_loss)
best_loss = np.array(best_loss)

In [12]:
# loss값 저장
np.save(PATH + 'train_loss_lstm_2layers_2048B_1', train_loss)
np.save(PATH + 'val_loss_lstm_2layers_2048B_1', val_loss)
np.save(PATH + 'best_loss_lstm_2layers_2048B_1', best_loss)

In [ ]:
test_input = np.load(LOADPATH + 'test_input_minmax.npy')
min_ = np.load(PATH + 'MinMax/MIN.npy')
max_ = np.load(PATH + 'MinMax/MAX.npy')

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
model = LSTM()
if platform.system() == "Darwin":
    model.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth', map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth'))

### Prediction

In [ ]:
prediction = inference(infer_model, test_loader, device)
prediction.shape

In [ ]:
def return_scaler(arr, min_ = min_, max_ = max_):
    denom = max_ - min_
    for i in tqdm(range(len(arr))):
        arr[i] = (arr[i]*denom[i]) + min_[i]
    
    return arr

prediction_unscale = np.around(return_scaler(prediction))
prediction_unscale

In [ ]:
submit = pd.read_csv(PATH + '/sample_submission.csv') # submit은 15690개 상품 / 예측은 15682개 상품
submit.iloc[:, 1:] = prediction_unscale
submit

In [ ]:
SAVE_SUBMIT = os.getcwd() + '/data/lstm_2Layers_0822.csv'
submit.to_csv(SAVE_SUBMIT, index_label=False)